Example to extract logical hierarchy from:

niet in behandeling nemen aanvraag verblijfsvergunning regulier bepaalde tijd

from file Interpretatie_Vw_over_besluiten_op_aanvragen_voor_een_verblijfsvergunning_regulier_bepaalde_tijd.json

In [1]:
# import networkx as nx
# import matplotlib.pyplot as plt
# from networkx.drawing.nx_pydot import graphviz_layout
import json

In [2]:
def gather_preconditions_parent(parent, preconditions_json, preconds_list, precond_texts):
    """
    Get all the preconditons of an act frame.


    """
    children = parent['children']
    for child in children:
        if 'frame' in child.keys():
            # print("This is happening at all, yay!")
            #TODO
            preconds_list.append(child['frame'])
            precond_texts[child['frame']] = ""
        gather_preconditions_parent(child, preconditions_json, preconds_list, precond_texts)
    
    return preconds_list, precond_texts

In [3]:
def extract_preconditions(file):
    """
    Extract mappings of all preconditions.pdf

    Returns:
    frames_precond (dict): mapping of  
    precond_texts, 
    actions_texts
    """

    # Example input data (JSON) as a Python dictionary
    # Open and load the JSON data from a file
    with open(file, 'r') as file:
        data = json.load(file)

    print(f"sourcedocs length: {len(data['frames'])}")

    frames = data['frames']
    frames_precond = {}
    precond_texts = {}
    actions_texts = {}

    # gather all the precondition IDs from the frame
    for frame in frames:
        
        if frame['typeId'] == 'act':
            
            preconditions = frame['precondition']
            if len(preconditions.get('children')) != 0:
                this_frame_preconds_list = []
                frames_precond[frame['id']], precond_texts = gather_preconditions_parent(frame['precondition'], preconditions, this_frame_preconds_list, precond_texts)
                actions_texts[frame['id']] = frame['act']
            elif preconditions.get('frame') is not None: # and len(preconditions.get('children')) == 0
                print(f"preconds: {preconditions.keys()}")
                print(f"frame id: {frame['id']}")
                frames_precond[frame['id']] = [preconditions['frame']]
                precond_texts.setdefault(preconditions['frame'], "")
                actions_texts[frame['id']] = frame['act']
            # else:
            #     this_frame_preconds_list = []
            #     frames_precond[frame['id']], precond_texts = gather_preconditions_parent(frame['precondition'], preconditions, this_frame_preconds_list, precond_texts)
            #     actions_texts[frame['id']] = frame['act']

            
            # precond_texts.setdefault(frames_precond[frame['id']], "")
    
    for frame in frames:
        # add actual text of the respective preconditions to the dictionary
        if frame['id'] in precond_texts.keys():
            if not frame['fact']:
                precond_texts[frame['id']] = frame['label']
            else:
                precond_texts[frame['id']] = frame['fact']

            
        


    return frames_precond, precond_texts, actions_texts

In [4]:
# file = 'Interpretatie_Vw_over_besluiten_op_aanvragen_voor_een_verblijfsvergunning_regulier_bepaalde_tijd.json'
# frames_precond, precond_texts, actions_texts = extract_preconditions(file)

# example = frames_precond["2168d545-ef9d-4d7f-b21f-ee3fa1b0d8df"]
# action_text = actions_texts["2168d545-ef9d-4d7f-b21f-ee3fa1b0d8df"]
# print(f"Keys: {frames_precond.keys()}")
# print(f"preconditiions: {len(example)}")
# print(f"Precondition texts: {precond_texts['4714e879-ad2c-4037-986a-9c1bb9388856']}")
# print(f"actions text: {action_text}")

# #TODO: somehow create a dataset from this...



In [5]:
# Add stuff to the JSON
# Function to create the desired JSON structure
def create_json_structure(file):
    frames_precond, precond_texts, action_text = extract_preconditions(file)
    result = []


    for act, preconditions in frames_precond.items():
        act_dict = {
            "file": file,
            "ID": act,
            "type": "act",
            "text": action_text[act],
            "precondition_texts": {precondition: precond_texts[precondition] for precondition in preconditions},
            "text_positions": {precondition: "" for precondition in preconditions}
            # "text_position": {pre["precondition_ID"]: pre["text_position"] for pre in act["preconditions"]}
        }
        result.append(act_dict)
    return result

# Create JSON structure
file_1 = 'Interpretatie_Vw_over_besluiten_op_aanvragen_voor_een_verblijfsvergunning_regulier_bepaalde_tijd.json'
file_2 = 'Participatiewet_most_recent_public.json'
file_3 = 'Besluit_vvr_bep_2024-11-06T14_53_09_interpretation.json'
file_4 = 'rijksbegrotingscyclus.json'
# file_2 = 
json_structure = create_json_structure(file_4)
# print(json_structure)


with open(f'acts_{file_4}.json', 'w') as file:
    json.dump(json_structure, file, indent=4)

sourcedocs length: 69
preconds: dict_keys(['frame', 'children', 'operatorToJoinChildren'])
frame id: 8d4d71e7-7e9f-4d40-964d-57fe362f94c4


In [6]:
# code to extract all actions in a for loop

ground_truth_1 = 'acts_Participatiewet_most_recent_public.json'

with open(ground_truth_1, 'r') as file:
        data = json.load(file)

for act in data:
        print(act['text'])

FileNotFoundError: [Errno 2] No such file or directory: 'acts_Participatiewet_most_recent_public.json'